# Clothes Classification

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [3]:
import os
current_path = os.getcwd()
print("當前路徑：", current_path)


當前路徑： /Users/huangyoucyuan/Documents/kaggle_competition/Clothes


In [4]:
tf.__version__

'2.16.1'

In [5]:
# DATA_DIR = "../input/clothing-dataset-full/images_original/"
DATA_DIR = "./Clothing_dataset_full/images_original/"
# DATA_RAW = "../input/clothing-dataset-full/images.csv"
DATA_RAW = "./Clothing_dataset_full/images.csv"
# MODEL_PATH = "../model.h5"
MODEL_PATH = "../model.h5"
WIDTH = 150
HEIGHT = 150
NUM_CHANNELS = 3
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 192
LEARNING_RATE = 1e-4
EPOCHS = 5

## Prepare Data

In [6]:
data = pd.read_csv(DATA_RAW)
data = data.sample(len(data))
data.head(10)

,image,sender_id,label,kids
1903,5bbe556b-67c3-434a-b91f-703cc7c0f453,48,T-Shirt,False
5254,7b6b898d-fb33-4cb3-bc01-7013feadebb7,204,T-Shirt,True
5334,5a9be8ea-9c5c-459b-9b5b-7adb4a0e2867,310,Outwear,False
4457,05814dc4-0520-47e6-91e0-402a971ba40a,204,Pants,False
928,8523dd6f-c10f-432f-a6fd-39239a89de54,367,Top,False
5307,2edb1db6-cd17-41d2-88e0-ba34801b656d,310,Outwear,False
3115,34fcfb9f-60ec-4cf2-b8cc-12ad05fbd3f1,54,Pants,False
1433,2dfbea36-6802-473c-b948-60a1e6181534,51,Shoes,False
5336,7d6fad20-bfca-42ec-8061-5e6a531a0938,204,Skirt,True
1257,e0dc1f06-78a4-488e-936a-5b2655a65ba6,92,Pants,False


In [7]:
data["image_path"] = DATA_DIR + data["image"] + ".jpg"
print(DATA_DIR+data["image"]+".jpg")
data.head()

1903    ./Clothing_dataset_full/images_original/5bbe55...
5254    ./Clothing_dataset_full/images_original/7b6b89...
5334    ./Clothing_dataset_full/images_original/5a9be8...
4457    ./Clothing_dataset_full/images_original/05814d...
928     ./Clothing_dataset_full/images_original/8523dd...
                              ...                        
4823    ./Clothing_dataset_full/images_original/e6230e...
782     ./Clothing_dataset_full/images_original/9253ed...
1880    ./Clothing_dataset_full/images_original/a81df0...
136     ./Clothing_dataset_full/images_original/c1b10b...
2478    ./Clothing_dataset_full/images_original/da574a...
Name: image, Length: 5403, dtype: object


,image,sender_id,label,kids,image_path
1903,5bbe556b-67c3-434a-b91f-703cc7c0f453,48,T-Shirt,False,./Clothing_dataset_full/images_original/5bbe55...
5254,7b6b898d-fb33-4cb3-bc01-7013feadebb7,204,T-Shirt,True,./Clothing_dataset_full/images_original/7b6b89...
5334,5a9be8ea-9c5c-459b-9b5b-7adb4a0e2867,310,Outwear,False,./Clothing_dataset_full/images_original/5a9be8...
4457,05814dc4-0520-47e6-91e0-402a971ba40a,204,Pants,False,./Clothing_dataset_full/images_original/05814d...
928,8523dd6f-c10f-432f-a6fd-39239a89de54,367,Top,False,./Clothing_dataset_full/images_original/8523dd...


In [8]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT
)
train_set = generator.flow_from_dataframe(
    dataframe=data,
    x_col="image_path",
    y_col="label",
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_set = generator.flow_from_dataframe(
    dataframe=data,
    x_col="image_path",
    y_col="label",
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)
num_classes = len(train_set.class_indices)
print("Total classes:", num_classes)

Found 4323 validated image filenames belonging to 20 classes.
Found 1080 validated image filenames belonging to 20 classes.
Total classes: 20


## Design model

In [9]:
model = keras.Sequential([
    keras.Input(shape=(WIDTH, HEIGHT, NUM_CHANNELS)),
    keras.layers.Conv2D(128, (5,5), activation=tf.nn.relu),
    keras.layers.Conv2D(128, (5,5), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (5,5), activation=tf.nn.relu),
    keras.layers.Conv2D(64, (5,5), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (5,5), activation=tf.nn.relu),
    keras.layers.Conv2D(32, (5,5), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 146, 146, 128)  │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 142, 142, 128)  │       409,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 71, 71, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 67, 67, 64)     │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 63, 63, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 31, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 27, 27, 32)     │        51,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 23, 23, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 11, 11, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3872)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       495,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,301,972 (4.97 MB)

 Trainable params: 1,301,972 (4.97 MB)

 Non-trainable params: 0 (0.00 B)

## Train model

In [10]:
train_set

In [11]:

model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=LEARNING_RATE),
              metrics=["accuracy"])

model.fit(train_set, epochs=EPOCHS)

/Users/huangyoucyuan/Documents/kaggle_competition/kaggle/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-05-23 20:17:55.215445: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3 of 8
2024-05-23 20:18:13.848014: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 7 of 8
2024-05-23 20:18:18.866619: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


## Evaluate model

In [ ]:
model.evaluate(val_set)

## Save model

In [ ]:
model.save(MODEL_PATH)

In [ ]:
from keras.models import load_model

model = load_model("path_to_your_model.h5")


In [ ]:
import pandas as pd

# 讀取包含圖片路徑的資料表
TEST_DIR = "./Clothing_dataset_full/test/"

data = pd.read_csv(TEST_DIR + "test.csv")


In [ ]:
from keras.preprocessing import image
import numpy as np

predictions = []

for index, row in data.iterrows():
    img_path = row["image"]  # 獲取圖片路徑
    img = image.load_img(img_path, target_size=(WIDTH, HEIGHT))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # 正規化

    # 進行預測
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    
    predictions.append(predicted_class)

# 將預測結果添加到資料表中
data["predicted_label"] = predictions


In [ ]:
print(data)